***GENERATED CODE FOR lastpaymentmanualapp PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DiagnosisCodeID1', 'threshold': 40415.89232883203, 'transformation_label': 'Binarizer'}], 'feature': 'DiagnosisCodeID1', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '105525', 'mean': '40415.89', 'stddev': '25478.64', 'min': '13472.0', 'max': '94919.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'DiagnosisCodeID1'}, {'feature_label': 'DiagnosisCodeID1', 'threshold': 40415.89232883203, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('DiagnosisCodeID1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run lastpaymentmanualappHooks.ipynb
try:
	#sourcePreExecutionHook()

	dbfs = DBFSConnector.fetch(spark, "{'url': '/Whitespace/whitespace_days_to_first_last_decision_cleaned.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(dbfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run lastpaymentmanualappHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(dbfs,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "_c0", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "264405.23", "stddev": "152932.94", "min": "6", "max": "530350", "missing": "0"}, "updatedLabel": "_c0"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "first_decision", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "105525", "mean": "15.44", "stddev": "9.3", "min": "1.0", "max": "194.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "first_decision"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "last_decision", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "105525", "mean": "28.76", "stddev": "27.15", "min": "1.0", "max": "205.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "last_decision"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CPTCodeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "1817.91", "stddev": "1214.05", "min": "1", "max": "4914", "missing": "0"}, "updatedLabel": "CPTCodeID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FacilityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "102.13", "stddev": "48.49", "min": "0", "max": "179", "missing": "0"}, "updatedLabel": "FacilityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ProviderID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "134.12", "stddev": "74.38", "min": "0", "max": "263", "missing": "0"}, "updatedLabel": "ProviderID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PayerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "142.4", "stddev": "53.83", "min": "0", "max": "251", "missing": "0"}, "updatedLabel": "PayerId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DepartmentID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "19.71", "stddev": "11.56", "min": "0", "max": "50", "missing": "0"}, "updatedLabel": "DepartmentID"}, {"transformationsData": [{"feature_label": "DiagnosisCodeID1", "threshold": 40415.89232883203, "transformation_label": "Binarizer"}], "feature": "DiagnosisCodeID1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "105525", "mean": "40415.89", "stddev": "25478.64", "min": "13472.0", "max": "94919.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "LoadYear"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "7.0", "stddev": "0.0", "min": "7", "max": "7", "missing": "0"}, "updatedLabel": "LoadMonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "28.05", "stddev": "0.34", "min": "28", "max": "31", "missing": "0"}, "updatedLabel": "LoadWeek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "9.38", "stddev": "2.34", "min": "9", "max": "29", "missing": "0"}, "updatedLabel": "LoadDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "3.0", "stddev": "0.38", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "LoadDayofweek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LoadDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "191.38", "stddev": "2.34", "min": "191", "max": "211", "missing": "0"}, "updatedLabel": "LoadDayofyear"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "2019.97", "stddev": "0.18", "min": "2017", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfServiceYear"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "3.05", "stddev": "2.11", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "DateOfServiceMonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "11.39", "stddev": "8.32", "min": "1", "max": "52", "missing": "0"}, "updatedLabel": "DateOfServiceWeek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "15.67", "stddev": "8.61", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfServiceDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "1.95", "stddev": "1.43", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "DateOfServiceDayofweek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfServiceDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "77.79", "stddev": "65.36", "min": "1", "max": "365", "missing": "0"}, "updatedLabel": "DateOfServiceDayofyear"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "DateOfEntryYear"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "2.81", "stddev": "1.43", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "DateOfEntryMonth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "10.97", "stddev": "6.28", "min": "1", "max": "22", "missing": "0"}, "updatedLabel": "DateOfEntryWeek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "16.06", "stddev": "8.58", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfEntryDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDayofweek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "2.09", "stddev": "1.52", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "DateOfEntryDayofweek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DateOfEntryDayofyear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "105525", "mean": "70.91", "stddev": "43.98", "min": "1", "max": "152", "missing": "0"}, "updatedLabel": "DateOfEntryDayofyear"}, {"feature": "DiagnosisCodeID1_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "105525", "mean": "0.29", "stddev": "0.45", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1_binarize..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run lastpaymentmanualappHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["_c0", "first_decision", "CPTCodeID", "FacilityID", "ProviderID", "PayerId", "DepartmentID", "DiagnosisCodeID1_binarizer", "LoadYear", "LoadMonth", "LoadWeek", "LoadDay", "LoadDayofweek", "LoadDayofyear", "DateOfServiceYear", "DateOfServiceMonth", "DateOfServiceWeek", "DateOfServiceDay", "DateOfServiceDayofweek", "DateOfServiceDayofyear", "DateOfEntryYear", "DateOfEntryMonth", "DateOfEntryWeek", "DateOfEntryDay", "DateOfEntryDayofweek", "DateOfEntryDayofyear"], "last_decision")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
